In [1]:
import pandas as pd
import numpy as np


In [4]:
data=pd.read_csv("../data/exoplanet_dataset.csv")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5599 entries, 0 to 5598
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   P_NAME                5599 non-null   object 
 1   P_DETECTION           5599 non-null   object 
 2   P_DISCOVERY_FACILITY  5599 non-null   object 
 3   P_YEAR                5599 non-null   int64  
 4   P_UPDATE              5599 non-null   object 
 5   P_MASS                5592 non-null   float64
 6   P_MASS_ORIGIN         5599 non-null   object 
 7   P_RADIUS              5592 non-null   float64
 8   P_PERIOD              5350 non-null   float64
 9   P_SEMI_MAJOR_AXIS     5595 non-null   float64
 10  P_ECCENTRICITY        4822 non-null   float64
 11  P_INCLINATION         4288 non-null   float64
 12  P_OMEGA               1659 non-null   float64
 13  S_NAME                5599 non-null   object 
 14  S_NAME_HD             971 non-null    object 
 15  S_NAME_HIP           

In [6]:

#these the earth like planet
data[data['P_HABITABLE'] == 1]

,P_NAME,P_DETECTION,P_DISCOVERY_FACILITY,P_YEAR,P_UPDATE,P_MASS,P_MASS_ORIGIN,P_RADIUS,P_PERIOD,P_SEMI_MAJOR_AXIS,...,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,S_CONSTELLATION,S_CONSTELLATION_ABR,S_CONSTELLATION_ENG
480,TOI-700 e,Transit,Transiting Exoplanet Survey Satellite (TESS),2023,2024-02-16,0.818,M-R relationship,0.953,27.809780,0.134000,...,0.408662,0.028274,0.286631,1,0,Warm,1,Dorado,Dor,Goldfish
537,TOI-700 d,Transit,Transiting Exoplanet Survey Satellite (TESS),2020,2024-02-16,1.250,M-R relationship,1.073,37.423960,0.163300,...,0.408662,0.028274,0.286631,1,1,Warm,1,Dorado,Dor,Goldfish
762,LP 890-9 c,Transit,SPECULOOS Southern Observatory,2022,2022-09-14,25.300,Mass,1.367,8.457463,0.039840,...,0.102533,0.006541,0.245193,1,1,Warm,1,Eridanus,Eri,River
934,Teegarden's Star b,Radial Velocity,Calar Alto Observatory,2019,2024-03-05,1.160,Msini,1.050,4.906340,0.025900,...,0.072588,0.008444,0.234823,1,0,Warm,1,Aries,Ari,Ram
935,Teegarden's Star c,Radial Velocity,Calar Alto Observatory,2019,2024-03-05,1.050,Msini,1.020,11.416000,0.045500,...,0.072588,0.008444,0.234823,1,1,Warm,1,Aries,Ari,Ram
1291,Kepler-1649 c,Transit,Kepler,2020,2020-04-16,1.200,M-R relationship,1.060,19.535270,0.064900,...,0.194028,0.015156,0.269791,1,0,Warm,1,Cygnus,Cyg,Swan
1977,TRAPPIST-1 g,Transit,Multiple Observatories,2017,2022-11-14,1.321,Mass,1.129,12.352446,0.046830,...,0.063513,0.003648,0.142824,1,1,Warm,1,Aquarius,Aqr,Water Carrier
2239,TRAPPIST-1 d,Transit,La Silla Observatory,2016,2022-11-14,0.388,Mass,0.788,4.049219,0.022270,...,0.063513,0.003648,0.142824,1,0,Warm,1,Aquarius,Aqr,Water Carrier
2267,GJ 1002 b,Radial Velocity,Multiple Observatories,2022,2022-12-17,1.080,Msini,1.030,10.346500,0.045700,...,0.101243,0.008820,0.227551,1,1,Warm,1,Cetus,Cet,Whale
2412,GJ 667 C e,Radial Velocity,La Silla Observatory,2013,2014-05-14,2.700,Msini,1.450,62.240000,0.213000,...,0.316129,0.021635,0.277482,1,1,Warm,1,Scorpius,Sco,Scorpion


In [ ]:
useless_features = [
    'P_NAME', 'P_DETECTION', 'P_DISCOVERY_FACILITY', 'P_YEAR', 'P_UPDATE',
    'P_MASS_ORIGIN', 'S_NAME', 'S_NAME_HD', 'S_NAME_HIP', 'S_RA', 'S_DEC',
    'S_RA_STR', 'S_DEC_STR', 'S_RA_TXT', 'S_DEC_TXT', 'S_CONSTELLATION',
    'S_CONSTELLATION_ABR', 'S_CONSTELLATION_ENG'
]